Stokes Equation
===

The Stokes equation is the basic model for incompressible fluids. While several terms of the advanced Navier-Stokes equations are skipped, the Stokes equation allows to study the difficulty of incompressibility constraints.

The unknown fields are 

\begin{eqnarray*}
\text{velocity} & & u : \Omega \rightarrow R^d \\
\text{pressure} & & p : \Omega \rightarrow R
\end{eqnarray*}

The Stokes equation is a system of equations

$$
\begin{array}{ccccl}
-\Delta u & + & \nabla p & = & f \\
\operatorname{div} u & & & = & 0
\end{array}
$$

with a given force density $f : \Omega \rightarrow R^2$. The first equation models balance of momentum, the second one is the incompressibility constraint.
In the simplest case, we set boundary conditions $u = 0$ on $\partial \Omega$




Variational Formulation
----

The weak form is : Find $u \in V := [H_0^1(\Omega)]^d$ and $p \in Q:=  L_2^0(\Omega)$ such that

$$
\begin{array}{ccccll}
\int \nabla u \nabla v & + & \int \operatorname{div} v p & = & \int f v & \forall \, v \in V \\
\int \operatorname{div} u q & & & = & 0 & \forall \, q \in Q
\end{array}
$$

The pressure is defined up to a constant, and the divergence equation is  satisfied for constant test-functions, anyway. Thus the space $L_2^0$ of $L_2$-functions with zero mean value.

Finite Element Spaces
---
We discretize the Stokes equation by different finite element spaces. For the velocity we use continuous, $H^1$-conforming elements. For the pressure, which is a field in $L_2$, discontinuous finite elements are natural. But, alternatively, one may also use continuous elements.

We try different combinations of finite element spaces for $V$ and $Q$.

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)

Use continuous elements of order $k=2$ for each velocity component, and piece-wise constants for pressure:

In [2]:
V = VectorH1(mesh, order=2, dirichlet=".*")
Q = L2(mesh, order=0)
X = FESpace([V,Q])

In [5]:
u,p = X.TrialFunction()
v,q = X.TestFunction()
a = BilinearForm(X)
a += SymbolicBFI(InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p-1e-10*p*q)
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI( (x-0.5)*v[1])
f.Assemble()

gfu = GridFunction(X)
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Draw (gfu.components[0], mesh, "velocity")
Draw (gfu.components[1], mesh, "pressure")

Try a $P^2 \times P^{1,dc}$ pairing:

In [4]:
V = VectorH1(mesh, order=2, dirichlet=".*")
Q = L2(mesh, order=1)
X = FESpace([V,Q])
# go back and solve again

$P^{2+} \times P^{1,dc}$, i.e. adding cubic bubbles:

In [ ]:
V = VectorH1(mesh, order=2, orderinner=3, dirichlet=".*")
Q = L2(mesh, order=1)
X = FESpace([V,Q])
# go back and solve again

The Taylor Hood element $P^2 \times P^1$ with continuous pressure:

In [ ]:
V = VectorH1(mesh, order=2, dirichlet=".*")
Q = H1(mesh, order=1)
X = FESpace([V,Q])
# go back and solve again

We see that the pressure space must not be too rich, otherwise we obtain bad solutions, in particular for the pressure. 